In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
# forex_data = pd.read_csv('/content/drive/MyDrive/BTP Project files/forex1.csv', encoding=encoding)
forex_data = pd.read_csv('/content/drive/MyDrive/NLP BTP/sentiment_annotated_with_texts.csv')
forex_data.head()

,published_at,ticker,true_sentiment,title,author,url,source,text,finbert_sentiment,finbert_sent_score
0,2023-01-12 07:47:00,EURCHF,Positive,Euro to benefit from the ECBs pronounced hawki...,FXStreet Insights Team,https://www.fxstreet.com/news/euro-to-benefit-...,FX Street,The Euro was able to appreciate particularly s...,Positive,0.85
1,2023-01-12 10:34:00,EURCHF,Positive,EURCHF Trend higher may remain in place – ING,FXStreet Insights Team,https://www.fxstreet.com/news/eur-chf-trend-hi...,FX Street,EUR/CHF yesterday broke above 1.00. Economists...,Positive,0.51
2,2023-01-12 11:40:00,EURCHF,Neutral,Does a jump in EURCHF point to a break above 1...,FXStreet Insights Team,https://www.fxstreet.com/news/does-a-jump-in-e...,FX Street,EUR/CHF vaults parity for the first time since...,Neutral,0.37
3,2023-01-12 15:32:00,EURCHF,Positive,EURCHF could extend its advance back to levels...,FXStreet Insights Team,https://www.fxstreet.com/news/eur-chf-could-ex...,FX Street,EUR/CHF climbs back above parity. Economists a...,Positive,0.64
4,2023-01-13 11:37:00,EURCHF,Positive,EURCHF to head higher towards 10130 and projec...,FXStreet Insights Team,https://www.fxstreet.com/news/eur-chf-to-head-...,FX Street,EUR/CHF has broken out above the sideways rang...,Positive,0.83


In [ ]:

import pandas as pd
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/NLP BTP/sentiment_annotated_with_texts.csv')

# Count the number of occurrences of each sentiment category
sentiment_counts = df['true_sentiment'].value_counts()

# Display the counts
print(sentiment_counts)


true_sentiment
Neutral     815
Positive    767
Negative    709
Name: count, dtype: int64


In [ ]:
import pandas as pd

sentiment_counts = df['Sentiment'].value_counts()

# Display the counts
print(sentiment_counts)


Sentiment
Positive    6
Neutral     2
Negative    2
Name: count, dtype: int64


In [ ]:
transformed_df.shape

(2291, 2)

In [ ]:
X = transformed_df['Text'].to_list()
y = transformed_df['Sentiment'].to_list()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
preds = []
preds_proba = []
tokenizer_kwargs = {"padding": True, "truncation": True, "max_length": 512}

for x in X:
    with torch.no_grad():
        input_sequence = tokenizer(x, return_tensors="pt", **tokenizer_kwargs)
        logits = model(**input_sequence).logits
        scores = {
            k: v
            for k, v in zip(
                model.config.id2label.values(),
                scipy.special.softmax(logits.numpy().squeeze()),
            )
        }
        sentimentFinbert = max(scores, key=scores.get)
        probabilityFinbert = max(scores.values())
        preds.append(sentimentFinbert)
        preds_proba.append(probabilityFinbert)


print(preds)
print(preds_proba)
print(y)


['positive', 'positive', 'neutral', 'positive', 'positive', 'positive', 'positive', 'positive', 'neutral', 'negative', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'neutral', 'negative', 'neutral', 'positive', 'positive', 'negative', 'positive', 'negative', 'neutral', 'neutral', 'positive', 'positive', 'neutral', 'positive', 'neutral', 'negative', 'negative', 'negative', 'neutral', 'positive', 'neutral', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'negative', 'neutral', 'neutral', 'negative', 'negative', 'positive', 'neutral', 'positive', 'neutral', 'negative', 'neutral', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'neutral', 'positive', 'neutral', 'positive', 'neutral', 'negative', 'negative', 'neutral', 'negative', 'positive', 'positive', 'negative', 'neutral', 'negative', 'positive', 'neutral', 'positive', 

In [ ]:
preds_capitalized = [' '.join(word.capitalize() for word in sentiment.split()) for sentiment in preds]

print(f'Accuracy-Score: {accuracy_score(y, preds_capitalized)}')


Accuracy-Score: 0.5617634220864252
